In [2]:
import numpy as np #gereki oan kütüphaneleri (paketleri) içe aktarıyoruz
import cv2

CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat", #sınıfları yani etiketleri tanımlıyoruz
	"bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
	"dog", "horse", "motorbike", "person", "pottedplant", "sheep",
	"sofa", "train", "tvmonitor"]

COLORS = np.random.uniform(0,255,size=(len(CLASSES),3)) #sınıf sayısı kadar renk üretiyoruz

net = cv2.dnn.readNetFromCaffe("MobileNetSSD_deploy.prototxt.txt", "MobileNetSSD_deploy.caffemodel") #önceden eğitilmiş MobileNet SSD modelini yükleriz

cap = cv2.VideoCapture(0) #webcamden görüntü alıyoruz
cap.set(3,800)  #webcam den alınan görüntünün width ve height değerlerini değiştiriyoruz
cap.set(4,600)
        
while True: #frameleri okumak için sonsuz bir döngü oluşturuyoruz
    
    success, frame = cap.read()   #frameeri okuyoruz
    (h,w) = frame.shape[:2] #framelerim height ve width değerlerini alıyoruz
    blob = cv2.dnn.blobFromImage(cv2.resize(frame,(300, 300)), 0.007843,(300, 300), 127.5)  #ssd 300x300 kabul ediyor bu yuzden yeniden boyutlandırıyoruz diğerleri ise modelle ilgili parametreler (ortalama değer çıkarımı yapılır)
    
    net.setInput(blob)
    detections = net.forward() #nesne algılama sonuçları detections değişkenine atılır
    
    for j in np.arange(0, detections.shape[2]):  #her bir algılanan nesne için
        
        confidence = detections[0,0,j,2]
        
        if confidence > 0.10: #eğer ki olasılık(tahmin) değeri 0.10'den buyukse
            
            idx = int(detections[0,0,j,1]) #algılanan nesnenin sınıf indexi hesaplanır
            box = detections[0,0,j,3:7]*np.array([w,h,w,h]) #x ,y ,width ve height değerleri hesaplanır
            (startX, startY, endX, endY) = box.astype("int") #box değişkenindeki koordinatlar int'e dönüştürdükten sonra (startX, startY, endX, endY) değişkenlerine eşitlenir
            
            label = "{}: {}".format(CLASSES[idx], confidence) #tespit edilen nesnelerin sınıfı ve tahmin değeri label değişkeninde
            cv2.rectangle(frame, (startX, startY), (endX, endY), COLORS[idx],2) #bulunan koordinatlarda image üzerine nesnelerin çevresine dikdörtgen(sınırlayıcı kutu) çizeriz
            y = startY - 16 if startY -16 >15 else startY + 16 #absürt bir konumda bilgilendirme olmaması adına
            cv2.putText(frame, label, (startX,y),cv2.FONT_HERSHEY_SIMPLEX, 0.5,COLORS[idx],2) #frame'imizin üstüne labeldeki verileri yazdırırız 
            
    cv2.imshow("ssd",frame) #ve en son ssd modeli ile tespit edilmiş nesneleri imshowlarız
    if cv2.waitKey(1)== ord("q"):  #eğer ki 'q' ya basılmışsa döngüyü kır
        break
cv2.destroyAllWindows() #ve tüm pencereleri kapat